# Using FABRIC Xilinx FPGA U280

Your compute nodes can include FPGAs. These devices are made available as FABRIC components and can be added to your nodes like any other component. Your project must have Component.FPGA permission tag in order to be able to provision them. 

This example notebook will demonstrate how to reserve and use a single Xilinx FPGA device on FABRIC. Note that programming FPGAs is complex and requires a licensed toolchain from Xilinx. FABRIC staff cannot help you in obtaining tool licenses - you (or your professor/research lead) must either purchase these tools from Xilinx or register with the [AMD (formerly Xilinx) University Progam (AUP or XUP)](https://www.xilinx.com/support/university.html) and request a donation.

## Pre-requisites

While this example does not demonstrate how to develop or program for FPGAs you may want to execute the notebook in this folder that shows how to create/re-create a slice with a persistent volume where you can store Xilinx tool packages, build artifacts etc. Transferring them usually take a long time and it helps to have them inside of FABRIC.
- [Slice for storing FPGA Tools](fpga_tools_storage.ipynb)

## Setup the Experiment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Select a site with available FPGA

The cells below help you create a slice that contains a single node with an attached FPGA. 

In [ ]:
FPGA_CHOICE='FPGA_Xilinx_U280'

# don't edit - convert from FPGA type to a resource column name
# to use in filter lambda function below
choice_to_column = {
    "FPGA_Xilinx_U280": "fpga_u280_available",
}

column_name = choice_to_column.get(FPGA_CHOICE, "Unknown")
print(f'{column_name=}')

Give the slice and the node in it meaningful names.

In [ ]:
# name the slice and the node 
slice_name=f'My Simple FPGA Slice with {FPGA_CHOICE}'
node_name='fpga-node'

print(f'Will create slice "{slice_name}" with node "{node_name}"')

Use a lambda filter to figure out which site the node will go to.

In [ ]:
import random

# you can limit to one of the sites on this list (or use None)
#allowed_sites = ['MAX', 'INDI']
allowed_sites = None

fpga_sites_df = fablib.list_sites(output='pandas', quiet=True, filter_function=lambda x: x[column_name] > 0)
# note that list_sites with 'pandas' doesn't actually return a dataframe like doc sez, it returns a Styler 
# based on the dataframe
fpga_sites = fpga_sites_df.data['Name'].values.tolist()
print(f'All sites with FPGA available: {fpga_sites}')

if len(fpga_sites)==0:
    print('Warning - no sites with available FPGAs found')
else:
    if allowed_sites and len(allowed_sites) > 0:
        fpga_sites = list(set(fpga_sites) & set(allowed_sites))
    
    print('Selecting a site at random' + f'among {allowed_sites}' if allowed_sites else '')
    site = random.choice(fpga_sites)
    print(f'Preparing to create slice "{slice_name}" with node {node_name} in site {site}')

## Create a slice with a node with FPGA at desired site

Create the desired slice with a FPGA component. 

In [ ]:
# Create Slice. Note that by default submit() call will poll for 360 seconds every 10-20 seconds
# waiting for slice to come up. Normal expected time is around 2 minutes. 
slice = fablib.new_slice(name=slice_name)

# Add node with a 100G drive and a couple of CPU cores (default)
node = slice.add_node(name=node_name, site=site, disk=100)
node.add_component(model=FPGA_CHOICE, name='fpga1')

#Submit Slice Request
slice.submit();

## Get the Slice

Retrieve the node information and save the management IP addresses.

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show();

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
node = slice.get_node(node_name) 
node.show()

fpga = node.get_component('fpga1')
fpga.show();


## FPGA PCI and USB Devices

Run the command <code>lspci</code> to see your FPGA PCI device(s) and <code>lsusb</code> to see the JTAG-over-USB device. This shows the FPGA PCI devices (there could be more than one depending on the bitfile loaded). 


In [ ]:
command = "sudo dnf install -q -y pciutils usbutils"
stdout, stderr = node.execute(command)

print('Checking to see if Xilinx PCI device(s) are present')
command = "lspci | grep 'Xilinx'"
stdout, stderr = node.execute(command)

print('Checking to see if JTAG-over-USB is available')
command = "lsusb -d 0403:6011"
stdout, stderr = node.execute(command)

## Discussion

Note that you SHOULD NOT attempt to program the FPGA at this point unless previously discussed with FABRIC staff. Generally changing the bitfile may change the BAR sizes and PCI functions of the device, which in turn may cause the underlying Dell server to automatically reboot. The net effect of this is your actions may affect experiments of other users who may have VMs sharing the same server.

__It is only safe to change the bitfile of the FPGA you just provisioned if you know that the PCI functions and BAR sizes in the FPGA are the same as in the bitfile you are trying to load__ - in this case a simple reboot of the VM will activate that bitfile and your experiment can proceed.

## Cleanup Your Experiment

In [ ]:
fablib.delete_slice(slice_name)